In [2]:
import requests
import re
import os
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver

# 판결문 데이터 크롤링

## Chrome 열기

In [17]:
URL = 'https://www.law.go.kr/precSc.do?menuId=7&subMenuId=47&tabMenuId=213#licPrec216727'

driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(3)

driver.get(url=URL)

In [18]:
# 시작 페이지로 들어가기
start_page = driver.find_element_by_xpath('/html/body/div[4]/div[1]/div[2]/div/div[5]/div/table/tbody/tr[1]/td[2]/a')
start_page.send_keys('\n')

## 함수 정의

In [19]:
# 현재 페이지의 판결문 데이터 출력
def get_data(number):
    row = {}
    row['번호'] = number
    # 제목
    Main_subject = driver.find_element_by_xpath("/html/body/div[4]/div[2]/div[3]/div/div/h2")
    row['제목'] = Main_subject.text
    
    # 사건번호
    case_number = driver.find_element_by_xpath("/html/body/div[4]/div[2]/div[3]/div/div/div[1]")
    row['사건번호'] = case_number.text

    # 전체 내용
    contents = driver.find_element_by_xpath("/html/body/div[4]/div[2]/div[3]/div/div/div[2]")
    # 큰 헤더
    header4 = contents.find_elements_by_tag_name('h4')
    # 큰 헤더 내용
    if '전문' not in header4[0].get_attribute('textContent'):
        contents_text = contents.find_elements_by_class_name('pty4')
    # 전문 세부내용
    header5 = contents.find_elements_by_tag_name('h5')
    # 전문 세부내용의 내용
    contents_text2 = contents.find_elements_by_class_name('pty4_dep1')
        
    for i, header in enumerate(header4):
        # 전문이 아닌 header 데이터 저장(판시사항, 판결요지, 참고판례 등...)
        if '전문' not in header.get_attribute('textContent'):
            row[header.get_attribute('textContent').strip()] = contents_text[i].get_attribute('textContent').strip()
        else:
            # 전문데이터 한번에 저장
            text = ""
            for j, subheader in enumerate(header5):
                subheader = subheader.get_attribute('textContent').strip()
                # 잘못된 데이터 error 방지
                try:
                    t = contents_text2[j].get_attribute('textContent').strip()
                except:
                    t=''
                text += subheader + '\n' + t + '\n'
                # 주문과 이유는 추가로 저장
                if ("주 문" in subheader) or ("이 유" in subheader):
                    row[subheader] = t
            row[header.get_attribute('textContent')] = text
                
    return row

# 왼쪽 판결문 리스트 추출
def get_judgement_list():
    judgements = driver.find_element_by_class_name('left_list_bx')
    judgement_li = judgements.find_elements_by_tag_name("a")
    
    return judgement_li

# 다음 페이지로 이동
def go_to_next_page():
    check = False
    paging = driver.find_element_by_class_name('paging')
    page_li = paging.find_element_by_tag_name("ol")
    
    # 숫자로된 페이지 들어가기
    for page in page_li.find_elements_by_tag_name("li"):
        if check:
            page.find_element_by_tag_name('a').send_keys('\n')
            break
        if page.get_attribute('class') == 'on':
            check = True
    # 다돌았을 경우 페이지 넘기기(>), 가장 마지막일 경우 -1반환
    else:
        next_page = paging.find_elements_by_tag_name('a')
        if len(next_page) < 6:
            return -1
        else:
            next_page = next_page[-2]
            next_page.send_keys('\n')

## 실행

In [28]:
data = pd.DataFrame()
noData = False
error = {}

while not noData:
    try:
        # 좌측 판결문 리스트 받고 하니씩 클릭
        judgement_li = get_judgement_list()
        i=0
        while i < len(judgement_li):
            try:
                number = judgement_li[i].find_element_by_class_name('tx').get_attribute('textContent').split()[0]
                judgement_li[i].send_keys('\n')
                time.sleep(0.1)
                # 판결문 데이터 불러온 후 데이터 추가
                data = data.append(get_data(number), ignore_index=True)
                i += 1
            except selenium.common.exceptions.StaleElementReferenceException:
                print("URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>")
                continue
            except Exception as e:
                print("Error 발생: ", e)
                error[number] = e
                i += 1

        end = go_to_next_page()
        time.sleep(1.5)
        if end == -1: noData = True
            
    except selenium.common.exceptions.StaleElementReferenceException:
        print("URL 접근 오류 다시 접근중(StaleElementReferenceException) <position2>")
        continue

URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
Error 발생:  list index out of range
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
Error 발생:  list index out of range
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근중(StaleElementReferenceException) <position1>
URL 접근 오류 다시 접근

KeyboardInterrupt: 

In [91]:
data.head()

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목,【이 유(주1)】,【양 형 이 유】
0,1,상고이유를 판단한다. 1. 헌법불합치결정과 잠정적용의 범위구 형사소송법(2019...,【피 고 인】\n피고인\n【상 고 인】\n검사\n【변 호 인】\n법무법인 여는 담당...,상고를 기각한다.,"헌법 제16조, 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되...","대법원 2011. 9. 29. 선고 2008두18885 판결(공2011하, 2234...",(가) 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되기 전의 ...,헌법재판소가 구 형사소송법 제216조 제1항 제1호 중 제200조의2에 관한 부분(...,"[대법원 2021. 5. 27., 선고, 2018도13458, 판결]",특수공무집행방해치상ㆍ특수공무집행방해,NaN,NaN
1,2,상고이유를 판단한다. 1. 상고이유 제1점에 관하여 가. 「상가건물 임대차보호...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 명경 담당변호사 김재윤...",상고를 모두 기각한다. 상고비용은 피고가 부담한다.,"[1] 상가건물 임대차보호법 제10조 제1항 제1호, 제10조의8 [2] 민법 제1...",[1] 대법원 2014. 7. 24. 선고 2012다58975 판결 / [2] 대법...,[1] 상가건물 임대차보호법(이하 ‘상가임대차법’이라고 한다) 제10조의8은 임대인...,[1] 상가건물 임대차보호법의 적용을 받는 상가건물의 임대차기간 중 어느 때라도 차...,"[대법원 2021. 5. 13., 선고, 2020다255429, 판결]",건물명도(인도),NaN,NaN
2,3,상고이유를 판단한다. 1. 원고의 상고이유에 대하여 가. 이 사건 RG보험 ...,"【원고, 상고인 겸 피상고인】\n유한회사 좋은기업지배구조연구소 (소송대리인 법무법인...","상고를 모두 기각한다. 상고비용 중 원고의 상고로 인한 부분은 원고가 부담하고, 피...","상법 제403조 제1항, 제2항, 제3항","대법원 2010. 4. 15. 선고 2009다98058 판결, 대법원 2018. 2...","상법 제403조 제1항, 제2항, 제3항은 ‘발행주식 총수의 100분의 1 이상에 ...",상법 제403조 제2항에 따른 서면에 기재되어야 하는 ‘이유’에 책임추궁 대상 이사...,"[대법원 2021. 5. 13., 선고, 2019다291399, 판결]",손해배상(기),NaN,NaN
3,4,상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 범위...,"【원고, 상고인】\n원고 (소송대리인 법무법인 중추 담당변호사 김선흠 외 3인)\n...","원심판결을 파기하고, 사건을 서울고등법원에 환송한다.","[1] 민법 제105조, 제390조, 제543조, 제551조, 민사소송법 제288조...","[1] 대법원 1989. 4. 25. 선고 86다카1147, 1148 판결(공198...",[1] 계약이 합의에 따라 해제되거나 해지된 경우에는 상대방에게 손해배상을 하기로 ...,"[1] 계약이 합의에 따라 해제되거나 해지된 경우, 채무불이행으로 인한 손해배상을 ...","[대법원 2021. 5. 7., 선고, 2017다220416, 판결]",대여금,NaN,NaN
4,5,"상고이유를 판단한다. 1. 원심의 인정에 따르면, 이 사건 부동산에 관하여 ‘종...","【원고, 피상고인】\n대미륵봉심회 (소송대리인 법무법인 파랑 담당변호사 이형재 외 ...","원심판결을 파기하고, 사건을 대전고등법원에 환송한다.","민법 제214조, 부동산등기법 제52조 제1호, 민사소송법 제288조","대법원 1993. 10. 8. 선고 93다28867 판결(공1993하, 3050),...",등기명의인의 표시변경 또는 경정의 부기등기가 등기명의인의 동일성을 해치는 방법으로 ...,등기명의인의 표시변경 또는 경정의 부기등기에 의하여 부동산등기사항증명서상의 표시가 ...,"[대법원 2021. 5. 7., 선고, 2020다299214, 판결]",등기명의인표시변경등기말소,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
82310,82311,상고이유 제1점은 원심판결은 증거의 인정은 사실심리의 직권에 속하는 것이란 관념하에...,"【원고, 상고인】\n박종식\n【피고, 피상고인】\n윤엽\n【주 문】\n원판결은 파훼...",원판결은 파훼함. 본건을 광주지방심리에 차려함.,NaN,NaN,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실의 변존을 인정함은 사회통념에 ...,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실 변존의 주장은 모순아닌가나. ...,"[대법원 1948. 5. 27., 4280민상383]",토지건물이전등기,NaN,NaN
82311,82312,직권으로써 피고 이용오 사망으로 인한 피고의 소송수계의 적부를 심안컨대 기록에 편철...,"【원고, 상고인】\n\n【피고, 피상고인】\n\n【원심판결】\n서울고등법원 1948...",원판결을 파훼함. 본건을 서울고등법원에 차려함.,민사소송법 제208조,NaN,타인의 양자로 출계한 자는 그 생부의 상속인이라 할 수 없음으로 생부가 사망한 경우...,출계한 양자와 생부사망에 의한 소송수계의 적부,"[대법원 1948. 4. 12., 4281민상417]",토지소유권이전등기,NaN,NaN
82312,82313,상고이유 제2점은 원판결은 기 이유 모두에서 「심안컨대 본건 기록에 첨부된 원고의 ...,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법...",원판결을 파훼함. 본건을 서울지방법원에 차려함.,"민사소송법 제80조, 제137조, 제395조",NaN,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,가. 소송위임 흠결의 항변과 당해 이원의 인증나. 계약해제존재의 가항변에 대한 판단...,"[대법원 1948. 4. 7., 선고, 4281민상362, 선고]",부동산소유권이전등기,NaN,NaN
82313,82314,상고이유 제1점은 원판결은 서증의 의미를 오해하였거나 일반사회의 경험법칙을 무시한 ...,"【원고, 상고인】\n정석용\n【피고, 피상고인】\n송종순\n【원심판결】\n청주지방법...",원판결을 파훼함. 본건을 청주지방법원으로 차려함.,NaN,NaN,매매계약에서 잔대금의 지불을 받은 후에 이전등기의무를 행하기로 약정한 경우의 약지는...,잔대금 지불후에 이전등기절차를 이행하기로한 약지와 동시이행과의 관계,"[대법원 1948. 4. 2., 4281민상298]",토지건물소유권이전등기,NaN,NaN


In [31]:
error

{'19497.': IndexError('list index out of range'),
 '19525.': IndexError('list index out of range'),
 '20162.': IndexError('list index out of range')}

In [30]:
data.to_csv("Judgement_Data_crawling.csv", encoding='utf-8-sig')

# 클롤링된 데이터 확인

In [89]:
# 번호 column type을 int형으로 변경
data = pd.read_csv("Judgement_Data_crawling.csv", index_col=0)
data['번호'] = data['번호'].astype(np.int)
data.head()

,번호,【이 유】,【전문】,【주 문】,사건번호,제목,【참조조문】,【참조판례】,【판결요지】,【판시사항】
0,19251,1. 처분의 경위 가. 원고는 중화인민공화국(이하 ‘중국’이라 한다) 국적의 조...,"【원고, 항소인】\n\n【피고, 피항소인】\n법무부장관\n【제1심판결】\n서울행정법...",1. 제1심 판결을 취소한다. 2. 피고가 2008. 12. 24. 원고에 대하...,"[서울고등법원 2009. 10. 6., 선고, 2009누11135, 판결]",국적신청불허가처분취소,NaN,NaN,NaN,NaN
1,19252,1. 처분의 경위 가. 원고는 2001. 6. 29. 소외인과 공동으로 임의경매 절...,【원 고】\n\n【피 고】\n강동세무서장\n【변론종결】\n2009. 8. 18.\n...,1. 원고의 청구를 기각한다. 2. 소송비용은 원고의 부담으로 한다.,"[서울행정법원 2009. 10. 6., 선고, 2009구단4640, 판결]",양도소득세부과처분취소,NaN,NaN,NaN,NaN
2,19253,"1. 인용하는 부분 이 법원의 판결이유는, 제1심 판결의 “3. 당사자들의 주장 및...","【원고(선정당사자), 항소인】\n\n【피고, 피항소인】\n주식회사 동양레저 (소송대...",1. 원고(선정당사자)들의 피고에 대한 항소를 모두 기각한다. 2. 항소비용은 ...,"[서울고등법원 2009. 10. 1., 선고, 2008나11483, 판결]",골프회원권분양예약무효확인 등,NaN,NaN,NaN,NaN
3,19254,"1. 청구원인에 대한 판단 갑 1 내지 6, 8호증(각 가지번호 포함)의 각 기재에...","【원고, 피항소인】\n\n【피고, 항소인】\n\n【제1심판결】\n대전지방법원 천안지...",1. 제1심 판결의 지연손해금 부분 중 아래에서 지급을 명하는 금원을 초과하는 피고...,"[대전지방법원 2009. 10. 1., 선고, 2009나8657, 판결]",대여금,NaN,NaN,NaN,NaN
4,19255,"이 판결에 적을 이유는 제1심 판결의 이유와 같은바, 원고의 청구를 기각한 제1심 ...","【원고, 항소인】\n청주부영3단지(11차) 임차인대표회의 (소송대리인 법무법인 청풍...",1. 원고의 항소를 기각한다. 2. 항소비용은 보조참가로 인한 비용을 포함하여 ...,"[대전고등법원 2009. 10. 1., 선고, 2009누1509, 판결]",분양전환승인의취소,NaN,NaN,NaN,NaN


## 중복확인

In [33]:
data.duplicated(['번호']).value_counts()

False    2297
dtype: int64

## 누락된 데이터 확인

In [90]:
idxRange = pd.Index(np.arange(1, 82316))
idxRange.difference(data.번호)

Int64Index([], dtype='int64')

### 누락된 데이터 가져오기
* 82551번 페이지 -> 1646페이지에서 실행

In [58]:
d20162 = get_data(20162)
d20162.keys()

dict_keys(['번호', '제목', '사건번호', '【주 문】', '【이 유】', '【전문】'])

In [60]:
d19525 = get_data(19525)
d19525.keys()

dict_keys(['번호', '제목', '사건번호', '【주 문】', '【이 유】', '【전문】'])

In [61]:
d19497 = get_data(19497)
d19497.keys()

dict_keys(['번호', '제목', '사건번호', '【주 문】', '【이 유】', '【전문】'])

In [76]:
data = data.append(d20162, ignore_index=True)
data = data.append(d19525, ignore_index=True)
data = data.append(d19497, ignore_index=True)
data

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목,【이 유(주1)】,【양 형 이 유】
0,1,상고이유를 판단한다. 1. 헌법불합치결정과 잠정적용의 범위구 형사소송법(2019...,【피 고 인】\n피고인\n【상 고 인】\n검사\n【변 호 인】\n법무법인 여는 담당...,상고를 기각한다.,"헌법 제16조, 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되...","대법원 2011. 9. 29. 선고 2008두18885 판결(공2011하, 2234...",(가) 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되기 전의 ...,헌법재판소가 구 형사소송법 제216조 제1항 제1호 중 제200조의2에 관한 부분(...,"[대법원 2021. 5. 27., 선고, 2018도13458, 판결]",특수공무집행방해치상ㆍ특수공무집행방해,NaN,NaN
1,2,상고이유를 판단한다. 1. 상고이유 제1점에 관하여 가. 「상가건물 임대차보호...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 명경 담당변호사 김재윤...",상고를 모두 기각한다. 상고비용은 피고가 부담한다.,"[1] 상가건물 임대차보호법 제10조 제1항 제1호, 제10조의8 [2] 민법 제1...",[1] 대법원 2014. 7. 24. 선고 2012다58975 판결 / [2] 대법...,[1] 상가건물 임대차보호법(이하 ‘상가임대차법’이라고 한다) 제10조의8은 임대인...,[1] 상가건물 임대차보호법의 적용을 받는 상가건물의 임대차기간 중 어느 때라도 차...,"[대법원 2021. 5. 13., 선고, 2020다255429, 판결]",건물명도(인도),NaN,NaN
2,3,상고이유를 판단한다. 1. 원고의 상고이유에 대하여 가. 이 사건 RG보험 ...,"【원고, 상고인 겸 피상고인】\n유한회사 좋은기업지배구조연구소 (소송대리인 법무법인...","상고를 모두 기각한다. 상고비용 중 원고의 상고로 인한 부분은 원고가 부담하고, 피...","상법 제403조 제1항, 제2항, 제3항","대법원 2010. 4. 15. 선고 2009다98058 판결, 대법원 2018. 2...","상법 제403조 제1항, 제2항, 제3항은 ‘발행주식 총수의 100분의 1 이상에 ...",상법 제403조 제2항에 따른 서면에 기재되어야 하는 ‘이유’에 책임추궁 대상 이사...,"[대법원 2021. 5. 13., 선고, 2019다291399, 판결]",손해배상(기),NaN,NaN
3,4,상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 범위...,"【원고, 상고인】\n원고 (소송대리인 법무법인 중추 담당변호사 김선흠 외 3인)\n...","원심판결을 파기하고, 사건을 서울고등법원에 환송한다.","[1] 민법 제105조, 제390조, 제543조, 제551조, 민사소송법 제288조...","[1] 대법원 1989. 4. 25. 선고 86다카1147, 1148 판결(공198...",[1] 계약이 합의에 따라 해제되거나 해지된 경우에는 상대방에게 손해배상을 하기로 ...,"[1] 계약이 합의에 따라 해제되거나 해지된 경우, 채무불이행으로 인한 손해배상을 ...","[대법원 2021. 5. 7., 선고, 2017다220416, 판결]",대여금,NaN,NaN
4,5,"상고이유를 판단한다. 1. 원심의 인정에 따르면, 이 사건 부동산에 관하여 ‘종...","【원고, 피상고인】\n대미륵봉심회 (소송대리인 법무법인 파랑 담당변호사 이형재 외 ...","원심판결을 파기하고, 사건을 대전고등법원에 환송한다.","민법 제214조, 부동산등기법 제52조 제1호, 민사소송법 제288조","대법원 1993. 10. 8. 선고 93다28867 판결(공1993하, 3050),...",등기명의인의 표시변경 또는 경정의 부기등기가 등기명의인의 동일성을 해치는 방법으로 ...,등기명의인의 표시변경 또는 경정의 부기등기에 의하여 부동산등기사항증명서상의 표시가 ...,"[대법원 2021. 5. 7., 선고, 2020다299214, 판결]",등기명의인표시변경등기말소,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
82310,82314,상고이유 제1점은 원판결은 서증의 의미를 오해하였거나 일반사회의 경험법칙을 무시한 ...,"【원고, 상고인】\n정석용\n【피고, 피상고인】\n송종순\n【원심판결】\n청주지방법...",원판결을 파훼함. 본건을 청주지방법원으로 차려함.,NaN,NaN,매매계약에서 잔대금의 지불을 받은 후에 이전등기의무를 행하기로 약정한 경우의 약지는...,잔대금 지불후에 이전등기절차를 이행하기로한 약지와 동시이행과의 관계,"[대법원 1948. 4. 2., 4281민상298]",토지건물소유권이전등기,NaN,NaN
82311,82315,상고이유 제1점은 원판결은 피고가 본건 가대를 최초에는 즉 1945년 10월 9일에...,"【원고, 상고인】\n중앙재목주식회사 우대표자 취체역 김박일\n【피고, 피상고인】\n...",원판결을 파훼함. 본건을 서울지방심리원에 차려함.,NaN,NaN,회사소유의 부동산을 회사대표자인 개인이 계약당사자로서 매도하고 다시 회사대표자 자격...,회사소유 부동산을 회사대표자 개인이 계약당사자로서 매도하고 다시 회사대표자자격으로써...,"[대법원 1947. 3. 23., 4280민상278]",토지건물매매무효확인,NaN,NaN
82312,20162,,【피 고 인】\n\n【검 사】\n황현덕외 1인\n【변 호 인】\n법무법인 율촌 담당...,1. 피고인 1 피고인을 징역 3년 6월에 처한다. 이 판결 선고 전의 구금일수 ...,NaN,NaN,NaN,NaN,"[서울중앙지방법원 2009. 6. 4., 선고, 2008고합1413,2009고합87...",배상명령신청·특정경제범죄가중처벌등에관한법률위반(배임){피고인2에대한예비적죄명:특정경...,NaN,NaN
82313,19525,"1. 제1심 판결의 인용 이 법원이 이 사건에 관하여 설시할 이유는, 아래에서 추가...","【원고, 피항소인】\n\n【피고, 항소인】\n근로복지공단\n【제1심판결】\n대구지방...",1. 피고의 항소를 기각한다. 2. 항소비용은 피고가 부담한다.,NaN,NaN,NaN,NaN,"[대구고등법원 2009. 8. 21., 선고, 2008누2102, 판결]",간병료부당이득금결정등,NaN,NaN


In [18]:
# 중복 재확인
data.duplicated('번호').value_counts()

False    3826
dtype: int64

### 번호를 index로 변환

In [35]:
data = data.set_index('번호', drop=True).sort_index()
data.head()

,【이 유】,【전문】,【주 문】,사건번호,제목,【참조조문】,【참조판례】,【판결요지】,【판시사항】
번호,,,,,,,,,
19251,1. 처분의 경위 가. 원고는 중화인민공화국(이하 ‘중국’이라 한다) 국적의 조...,"【원고, 항소인】\n\n【피고, 피항소인】\n법무부장관\n【제1심판결】\n서울행정법...",1. 제1심 판결을 취소한다. 2. 피고가 2008. 12. 24. 원고에 대하...,"[서울고등법원 2009. 10. 6., 선고, 2009누11135, 판결]",국적신청불허가처분취소,NaN,NaN,NaN,NaN
19252,1. 처분의 경위 가. 원고는 2001. 6. 29. 소외인과 공동으로 임의경매 절...,【원 고】\n\n【피 고】\n강동세무서장\n【변론종결】\n2009. 8. 18.\n...,1. 원고의 청구를 기각한다. 2. 소송비용은 원고의 부담으로 한다.,"[서울행정법원 2009. 10. 6., 선고, 2009구단4640, 판결]",양도소득세부과처분취소,NaN,NaN,NaN,NaN
19253,"1. 인용하는 부분 이 법원의 판결이유는, 제1심 판결의 “3. 당사자들의 주장 및...","【원고(선정당사자), 항소인】\n\n【피고, 피항소인】\n주식회사 동양레저 (소송대...",1. 원고(선정당사자)들의 피고에 대한 항소를 모두 기각한다. 2. 항소비용은 ...,"[서울고등법원 2009. 10. 1., 선고, 2008나11483, 판결]",골프회원권분양예약무효확인 등,NaN,NaN,NaN,NaN
19254,"1. 청구원인에 대한 판단 갑 1 내지 6, 8호증(각 가지번호 포함)의 각 기재에...","【원고, 피항소인】\n\n【피고, 항소인】\n\n【제1심판결】\n대전지방법원 천안지...",1. 제1심 판결의 지연손해금 부분 중 아래에서 지급을 명하는 금원을 초과하는 피고...,"[대전지방법원 2009. 10. 1., 선고, 2009나8657, 판결]",대여금,NaN,NaN,NaN,NaN
19255,"이 판결에 적을 이유는 제1심 판결의 이유와 같은바, 원고의 청구를 기각한 제1심 ...","【원고, 항소인】\n청주부영3단지(11차) 임차인대표회의 (소송대리인 법무법인 청풍...",1. 원고의 항소를 기각한다. 2. 항소비용은 보조참가로 인한 비용을 포함하여 ...,"[대전고등법원 2009. 10. 1., 선고, 2009누1509, 판결]",분양전환승인의취소,NaN,NaN,NaN,NaN


## 최종 데이터 저장

In [36]:
data.to_csv("Judgement_Data.csv", encoding='utf-8-sig')

# 최종 데이터 확인

In [11]:
final=pd.read_csv("data/Judgement_Data.csv")
final

C:\Users\saeji\anaconda3\envs\NLP_project\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목,【이 유(주1)】,【양 형 이 유】
0,1,상고이유를 판단한다. 1. 헌법불합치결정과 잠정적용의 범위구 형사소송법(2019...,【피 고 인】\n피고인\n【상 고 인】\n검사\n【변 호 인】\n법무법인 여는 담당...,상고를 기각한다.,"헌법 제16조, 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되...","대법원 2011. 9. 29. 선고 2008두18885 판결(공2011하, 2234...",(가) 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되기 전의 ...,헌법재판소가 구 형사소송법 제216조 제1항 제1호 중 제200조의2에 관한 부분(...,"[대법원 2021. 5. 27., 선고, 2018도13458, 판결]",특수공무집행방해치상ㆍ특수공무집행방해,NaN,NaN
1,2,상고이유를 판단한다. 1. 상고이유 제1점에 관하여 가. 「상가건물 임대차보호...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 명경 담당변호사 김재윤...",상고를 모두 기각한다. 상고비용은 피고가 부담한다.,"[1] 상가건물 임대차보호법 제10조 제1항 제1호, 제10조의8 [2] 민법 제1...",[1] 대법원 2014. 7. 24. 선고 2012다58975 판결 / [2] 대법...,[1] 상가건물 임대차보호법(이하 ‘상가임대차법’이라고 한다) 제10조의8은 임대인...,[1] 상가건물 임대차보호법의 적용을 받는 상가건물의 임대차기간 중 어느 때라도 차...,"[대법원 2021. 5. 13., 선고, 2020다255429, 판결]",건물명도(인도),NaN,NaN
2,3,상고이유를 판단한다. 1. 원고의 상고이유에 대하여 가. 이 사건 RG보험 ...,"【원고, 상고인 겸 피상고인】\n유한회사 좋은기업지배구조연구소 (소송대리인 법무법인...","상고를 모두 기각한다. 상고비용 중 원고의 상고로 인한 부분은 원고가 부담하고, 피...","상법 제403조 제1항, 제2항, 제3항","대법원 2010. 4. 15. 선고 2009다98058 판결, 대법원 2018. 2...","상법 제403조 제1항, 제2항, 제3항은 ‘발행주식 총수의 100분의 1 이상에 ...",상법 제403조 제2항에 따른 서면에 기재되어야 하는 ‘이유’에 책임추궁 대상 이사...,"[대법원 2021. 5. 13., 선고, 2019다291399, 판결]",손해배상(기),NaN,NaN
3,4,상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 범위...,"【원고, 상고인】\n원고 (소송대리인 법무법인 중추 담당변호사 김선흠 외 3인)\n...","원심판결을 파기하고, 사건을 서울고등법원에 환송한다.","[1] 민법 제105조, 제390조, 제543조, 제551조, 민사소송법 제288조...","[1] 대법원 1989. 4. 25. 선고 86다카1147, 1148 판결(공198...",[1] 계약이 합의에 따라 해제되거나 해지된 경우에는 상대방에게 손해배상을 하기로 ...,"[1] 계약이 합의에 따라 해제되거나 해지된 경우, 채무불이행으로 인한 손해배상을 ...","[대법원 2021. 5. 7., 선고, 2017다220416, 판결]",대여금,NaN,NaN
4,5,"상고이유를 판단한다. 1. 원심의 인정에 따르면, 이 사건 부동산에 관하여 ‘종...","【원고, 피상고인】\n대미륵봉심회 (소송대리인 법무법인 파랑 담당변호사 이형재 외 ...","원심판결을 파기하고, 사건을 대전고등법원에 환송한다.","민법 제214조, 부동산등기법 제52조 제1호, 민사소송법 제288조","대법원 1993. 10. 8. 선고 93다28867 판결(공1993하, 3050),...",등기명의인의 표시변경 또는 경정의 부기등기가 등기명의인의 동일성을 해치는 방법으로 ...,등기명의인의 표시변경 또는 경정의 부기등기에 의하여 부동산등기사항증명서상의 표시가 ...,"[대법원 2021. 5. 7., 선고, 2020다299214, 판결]",등기명의인표시변경등기말소,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
82310,82311,상고이유 제1점은 원심판결은 증거의 인정은 사실심리의 직권에 속하는 것이란 관념하에...,"【원고, 상고인】\n박종식\n【피고, 피상고인】\n윤엽\n【주 문】\n원판결은 파훼...",원판결은 파훼함. 본건을 광주지방심리에 차려함.,NaN,NaN,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실의 변존을 인정함은 사회통념에 ...,가. 매매계약합의해제와 합의해제후 매매대금수수 양사실 변존의 주장은 모순아닌가나. ...,"[대법원 1948. 5. 27., 4280민상383]",토지건물이전등기,NaN,NaN
82311,82312,직권으로써 피고 이용오 사망으로 인한 피고의 소송수계의 적부를 심안컨대 기록에 편철...,"【원고, 상고인】\n\n【피고, 피상고인】\n\n【원심판결】\n서울고등법원 1948...",원판결을 파훼함. 본건을 서울고등법원에 차려함.,민사소송법 제208조,NaN,타인의 양자로 출계한 자는 그 생부의 상속인이라 할 수 없음으로 생부가 사망한 경우...,출계한 양자와 생부사망에 의한 소송수계의 적부,"[대법원 1948. 4. 12., 4281민상417]",토지소유권이전등기,NaN,NaN
82312,82313,상고이유 제2점은 원판결은 기 이유 모두에서 「심안컨대 본건 기록에 첨부된 원고의 ...,"【원고, 피상고인】\n안유길\n【피고, 상고인】\n최석희\n【원심판결】\n서울지방법...",원판결을 파훼함. 본건을 서울지방법원에 차려함.,"민사소송법 제80조, 제137조, 제395조",NaN,가. 소송위임장이 사문서인 경우에 당사자 일방이 그 소송위임의 흠결을 주장한 때에는...,가. 소송위임 흠결의 항변과 당해 이원의 인증나. 계약해제존재의 가항변에 대한 판단...,"[대법원 1948. 4. 7., 선고, 4281민상362, 선고]",부동산소유권이전등기,NaN,NaN
82313,82314,상고이유 제1점은 원판결은 서증의 의미를 오해하였거나 일반사회의 경험법칙을 무시한 ...,"【원고, 상고인】\n정석용\n【피고, 피상고인】\n송종순\n【원심판결】\n청주지방법...",원판결을 파훼함. 본건을 청주지방법원으로 차려함.,NaN,NaN,매매계약에서 잔대금의 지불을 받은 후에 이전등기의무를 행하기로 약정한 경우의 약지는...,잔대금 지불후에 이전등기절차를 이행하기로한 약지와 동시이행과의 관계,"[대법원 1948. 4. 2., 4281민상298]",토지건물소유권이전등기,NaN,NaN


## 특이 column 처리 (이유(주1), 양형이유)

In [12]:
final[~final['【이 유(주1)】'].isnull()]

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목,【이 유(주1)】,【양 형 이 유】
13831,13832,NaN,【피 고 인】\n\n【검 사】\n김주필 외 1인\n【변 호 인】\n법무법인(유한) ...,"1. 피고인 1 피고인을 판시 제1, 제2의 가. 죄에 대하여 징역 1년 및 자격정...",NaN,NaN,NaN,NaN,"[서울중앙지방법원 2011. 12. 22., 선고, 2009고합731,2011고합3...",국가보안법 위반(특수잠입·탈출)·국가보안법 위반(회합·통신등)(피고인2에대하여일부공...,이 유,NaN


In [13]:
final[~final['【양 형 이 유】'].isnull()]

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목,【이 유(주1)】,【양 형 이 유】
29125,29126,NaN,【피고인】\n\n【항소인】\n검사\n【검사】\n우승배\n【변호인】\n법무법인 호남종...,NaN,"형법 제20조, 제314조 제1항",NaN,피해자가 불특정 다수인의 통행로로 이용되어 오던 기존 통로의 일부 소유자인 피고인으...,피해자가 불특정 다수인의 통행로로 이용되어 오던 기존 통로의 일부 소유자인 피고인으...,"[전주지법 2005. 6. 17., 선고, 2005노317, 판결: 상고]",업무방해,NaN,"피고인은 자신의 잘못을 반성하고 있고, 피해자의 공사차량의 통행으로 인하여 휴게소 ..."
29138,29139,NaN,【피고인】\n\n【항소인】\n피고인\n【검사】\n우승배\n【변호인】\n변호사 최동배...,NaN,"[1] 형법 제40조, 근로기준법 제30조 제1항, 제107조 제2항, 제1...","[4] 대법원 1976. 4. 27. 선고 76도146 판결(집24-1, 형128)",[1] 사용자가 근로자들이 노동조합을 조직·가입하고 자신의 부당노동행위를 근로감독관...,[1] 사용자가 근로자들이 노동조합을 조직·가입하고 자신의 부당노동행위를 근로감독관...,"[전주지법 2005. 6. 14., 선고, 2005노195, 판결: 상고]",근로기준법위반·근로자참여및협력증진에관한법률위반·노동조합및노동관계조정법위반,NaN,피고인이 노동조합을 결성하였다는 이유로 부당하게 6명의 근로자를 해고하는 잘못을 저...
29366,29367,NaN,【피고인】\n\n【항소인】\n피고인\n【검사】\n정만진\n【변호인】\n변호사 전봉진...,NaN,"형법 제129조 제1항, 특정범죄가중처벌등에관한법률 제2조 제1항",NaN,"교통업무를 담당하는 경찰관이, 자신의 처가 가입한 인터넷 다단계 판매회사가 부도난 ...","교통업무를 담당하는 경찰관이, 자신의 처가 가입한 인터넷 다단계 판매회사가 부도난 ...","[서울고법 2005. 5. 13., 선고, 2004노2238, 판결: 상고]",특정범죄가중처벌등에관한법률위반(뇌물)·폭력행위등처벌에관한법률위반(야간·공동체포){인...,NaN,"피고인의 이 사건 범행은, 피고인이 광명경찰서 경비교통과 종합상황실 근무자로서 66..."
29437,29438,NaN,【피고인】\n\n【항소인】\n피고인 및 군검찰관\n【검사】\n정만진\n【변호인】\n...,NaN,[1] 형법 제356조[2] 형법 제356조 [3] 형법 제356조 [4] ...,"[6] 대법원 2001. 2. 23. 선고 99두5498 판결(공2001상, 78...",[1] 육군 군단장이 군단장 지휘활동비로 책정된 금액에다 군단장 비서실 운영비를 포...,[1] 육군 군단장이 군단장 지휘활동비로 책정된 금액에다 군단장 비서실 운영비를 포...,"[서울고법 2005. 4. 29., 선고, 2004노1506, 판결: 확정]",업무상횡령{일부변경된죄명:주위적으로특정범죄가중처벌등에관한법률위반(뇌물)·예비적으로업...,NaN,피고인이 육군 3군단장 재직시 정당한 회계업무 담당자인 관리참모를 배제하고 비서실장...
30606,30607,Ⅰ. 항소이유의 요지 1. 피고인 (항소이유서 제출기간이 지난 뒤에 피고인의 변호...,【피 고 인】\n송두율\n【항 소 인】\n피고인 및 검사\n【검 사】\n정점식외 1...,원심판결 중 유죄부분을 파기한다.피고인을 징역 3년에 처한다.원심판결 선고 전의 구...,NaN,NaN,NaN,NaN,"[서울고등법원 2004. 7. 21., 선고, 2004노827, 판결]",형질변경허가반려처분취소,NaN,"1. 피고인의 죄책의 중요성위에서 유죄로 인정한 바와 같이, 피고인은 1973. 9..."


* 2개 모두 전문안에 있는 내용으로 삭제

In [14]:
final = final.drop(['【이 유(주1)】', '【양 형 이 유】'], axis=1)
final.head()

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목
0,1,상고이유를 판단한다. 1. 헌법불합치결정과 잠정적용의 범위구 형사소송법(2019...,【피 고 인】\n피고인\n【상 고 인】\n검사\n【변 호 인】\n법무법인 여는 담당...,상고를 기각한다.,"헌법 제16조, 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되...","대법원 2011. 9. 29. 선고 2008두18885 판결(공2011하, 2234...",(가) 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되기 전의 ...,헌법재판소가 구 형사소송법 제216조 제1항 제1호 중 제200조의2에 관한 부분(...,"[대법원 2021. 5. 27., 선고, 2018도13458, 판결]",특수공무집행방해치상ㆍ특수공무집행방해
1,2,상고이유를 판단한다. 1. 상고이유 제1점에 관하여 가. 「상가건물 임대차보호...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 명경 담당변호사 김재윤...",상고를 모두 기각한다. 상고비용은 피고가 부담한다.,"[1] 상가건물 임대차보호법 제10조 제1항 제1호, 제10조의8 [2] 민법 제1...",[1] 대법원 2014. 7. 24. 선고 2012다58975 판결 / [2] 대법...,[1] 상가건물 임대차보호법(이하 ‘상가임대차법’이라고 한다) 제10조의8은 임대인...,[1] 상가건물 임대차보호법의 적용을 받는 상가건물의 임대차기간 중 어느 때라도 차...,"[대법원 2021. 5. 13., 선고, 2020다255429, 판결]",건물명도(인도)
2,3,상고이유를 판단한다. 1. 원고의 상고이유에 대하여 가. 이 사건 RG보험 ...,"【원고, 상고인 겸 피상고인】\n유한회사 좋은기업지배구조연구소 (소송대리인 법무법인...","상고를 모두 기각한다. 상고비용 중 원고의 상고로 인한 부분은 원고가 부담하고, 피...","상법 제403조 제1항, 제2항, 제3항","대법원 2010. 4. 15. 선고 2009다98058 판결, 대법원 2018. 2...","상법 제403조 제1항, 제2항, 제3항은 ‘발행주식 총수의 100분의 1 이상에 ...",상법 제403조 제2항에 따른 서면에 기재되어야 하는 ‘이유’에 책임추궁 대상 이사...,"[대법원 2021. 5. 13., 선고, 2019다291399, 판결]",손해배상(기)
3,4,상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 범위...,"【원고, 상고인】\n원고 (소송대리인 법무법인 중추 담당변호사 김선흠 외 3인)\n...","원심판결을 파기하고, 사건을 서울고등법원에 환송한다.","[1] 민법 제105조, 제390조, 제543조, 제551조, 민사소송법 제288조...","[1] 대법원 1989. 4. 25. 선고 86다카1147, 1148 판결(공198...",[1] 계약이 합의에 따라 해제되거나 해지된 경우에는 상대방에게 손해배상을 하기로 ...,"[1] 계약이 합의에 따라 해제되거나 해지된 경우, 채무불이행으로 인한 손해배상을 ...","[대법원 2021. 5. 7., 선고, 2017다220416, 판결]",대여금
4,5,"상고이유를 판단한다. 1. 원심의 인정에 따르면, 이 사건 부동산에 관하여 ‘종...","【원고, 피상고인】\n대미륵봉심회 (소송대리인 법무법인 파랑 담당변호사 이형재 외 ...","원심판결을 파기하고, 사건을 대전고등법원에 환송한다.","민법 제214조, 부동산등기법 제52조 제1호, 민사소송법 제288조","대법원 1993. 10. 8. 선고 93다28867 판결(공1993하, 3050),...",등기명의인의 표시변경 또는 경정의 부기등기가 등기명의인의 동일성을 해치는 방법으로 ...,등기명의인의 표시변경 또는 경정의 부기등기에 의하여 부동산등기사항증명서상의 표시가 ...,"[대법원 2021. 5. 7., 선고, 2020다299214, 판결]",등기명의인표시변경등기말소


## error 데이터 처리

In [229]:
f = open("error.txt")
numbers = f.read()
error_idx = list(map(int, numbers.split()))
print(error_idx)

[351, 589, 1124, 2065, 2821, 4865, 4983, 5367, 5571, 5857, 6294, 6721, 6782, 7078, 8127, 8645, 8891, 9148, 9471, 9940, 10284, 13478, 15500, 17249, 18698, 18782, 19497, 19525, 20162, 57203]


In [246]:
errorFullContent = final[final['번호'].isin(error_idx)][['【전문】', '사건번호']]
errorFullContent

,【전문】,사건번호
350,"【원고, 피항소인】\n별지와 같음 (소송대리인 법무법인(유한) 한별 외 2인)\n【...","[서울고등법원 2020. 8. 19., 선고, 2020누36498, 판결]"
588,"【원고, 항소인】\n한국전력공사(소송대리인 변호사 이경우)\n【피고, 피항소인】\n...","[부산고등법원 2020. 4. 23., 선고, 2019나54552, 판결]"
1123,"【원고(재심원고), 피항소인 겸 항소인】\n별지 원고 명단 기재와 같다. (소송대리...","[서울고법 2019. 4. 18., 선고, 2018재나20200, 판결 : 상고]"
2064,"【원고, 항소인】\n원고 1 외 1인(소송대리인 법무법인 대안 담당변호사 정기호)\...","[부산고등법원 2018. 7. 4., 선고, 2017나59164, 판결]"
2820,"【원고, 피항소인】\n한국철도공사 (소송대리인 법무법인(유한) 화우 외 1인)\n【...","[서울고등법원 2017. 12. 7., 선고, 2017누71385, 판결]"
4864,"【원고, 항소인】\n학교법인 근명학교 (소송대리인 법무법인 평안 담당변호사 권형기 ...","[서울고등법원 2016. 10. 5., 선고, 2016누35924, 판결]"
4982,"【피 고 인】\n\n【항 소 인】\n피고인들 및 검사\n【검 사】\n안형준(기소),...","[광주고등법원 2016. 9. 6., 선고, (전주)2016노50, 판결]"
5366,"【원고, 피항소인】\n한국토지주택공사 (소송대리인 법무법인 (유한) 해송 담당변호사...","[부산고등법원 창원재판부 2016. 6. 22., 선고, 2016누10278, 판결]"
5570,"【원고, 피항소인】\n원고 1 외 3인 (소송대리인 변호사 신영훈 외 1인)\n【피...","[서울고등법원 2016. 5. 11., 선고, 2015누31307, 판결]"
5856,"【원고, 피항소인 겸 부대항소인】\n\n【피고, 항소인 겸 부대피항소인】\n\n【제...","[부산고등법원 2016. 2. 3., 선고, 2015나52497, 판결]"


In [263]:
print(errorFullContent.iloc[-12]['【전문】'])

【원고, 항소인 겸 피항소인】

【피고, 피항소인 겸 항소인】

【제1심판결】
대구지방법원 2012. 8. 24. 선고 2011가단54492 판결
【변론종결】
2013. 12. 20.
【주 문】
1. 제1심판결을 다음과 같이 변경한다. 가. 피고는 원고 1에게 38,798,562원, 원고 2에게 1,500,000원, 원고 3에게 500,000원 및 위 각 돈에 대하여 2011. 6. 6.부터 2014. 2. 7.까지는 연 5%, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율로 계산한 돈을 지급하라. 나. 원고들의 나머지 청구를 각 기각한다. 2. 소송총비용 중 2/5는 원고들이, 나머지는 피고가 각 부담한다. 3. 제1의 가항은 가집행할 수 있다.
【청구취지 및 항소취지】
1. 청구취지 피고는 원고 1에게 65,943,940원, 원고 2에게 5,000,000원, 원고 3에게 1,000,000원 및 위 각 돈에 대하여 2011. 6. 6.부터 2013. 12. 17.자 청구취지 및 청구원인정정서 부본 송달일까지는 연 5%, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율로 계산한 돈을 지급하라(원고 1은 당심에서 소극적 손해와 적극적 손해 부분 및 지연손해금에 대한 청구취지를, 원고 2, 3은 지연손해금에 대한 청구취지를 각 감축하였다).2. 항소취지가. 원고 제1심판결을 다음과 같이 변경한다. 피고는 원고 1에게 83,386,019원, 원고 2에게 5,000,000원, 원고 3에게 1,000,000원 및 위 각 돈에 대하여 2011. 6. 6.부터 2012. 5. 21.자 청구취지 및 청구원인정정서 부본 송달일까지는 연 5%, 그 다음 날부터 다 갚는 날까지는 연 20%의 각 비율로 계산한 돈을 지급하라.나. 피고 제1심판결 중 피고 패소 부분을 취소하고, 그 취소 부분에 해당하는 원고들의 청구를 모두 기각한다.
【이 유】
1. 제1심판결의 인용 이 법원이 이 사건에 관하여 설시할 이유는, 제1심판결 2의 나항과 3항 및 4항을 아래와 같이 고

## 결측치 데이터 확인

In [15]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82315 entries, 0 to 82314
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   번호      82315 non-null  int64 
 1   【이 유】   68748 non-null  object
 2   【전문】    82273 non-null  object
 3   【주 문】   69099 non-null  object
 4   【참조조문】  69375 non-null  object
 5   【참조판례】  38465 non-null  object
 6   【판결요지】  62422 non-null  object
 7   【판시사항】  70650 non-null  object
 8   사건번호    82315 non-null  object
 9   제목      82297 non-null  object
dtypes: int64(1), object(9)
memory usage: 6.3+ MB


### 전문 결측치 데이터 확인

* 번호\
    32, 38, 39, 49, 60, 68, 69, 71, 73, 81, 85, 86, 87, 91, 94, 95, 99, 110, 118, 126, 136, 138, 140, 162, 315, 530, 541, 646, 685, 1353, 2981, 9411, 26923, 32904, 41994(실제 전문 없음), 42075(실제 전문 없음), 68601, 71600, 79765, 80281, 80711, 81192

In [23]:
noFullContentNum = np.array(final[final['【전문】'].isnull()]['번호'])
noFullContentNum

array([   32,    38,    39,    49,    60,    68,    69,    71,    73,
          81,    85,    86,    87,    91,    94,    95,    99,   110,
         118,   126,   136,   138,   140,   162,   315,   530,   541,
         646,   685,  1353,  2981,  9411, 26923, 32904, 41994, 42075,
       68601, 71600, 79765, 80281, 80711, 81192], dtype=int64)

In [154]:
# 확인하면서 지우기
final[final['【전문】'].isnull()]

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목


In [156]:
# 현재 페이지의 글에서 전문부분 찾기
fullContent = driver.find_element_by_xpath("/html/body/div[4]/div[2]/div[3]/div/div/div[2]").get_attribute("textContent")
fullContent_start_idx = fullContent.find('【전문】')
add = fullContent[fullContent_start_idx+4:]
add

' [신 청 인] 신청인\n     [원 심] 제천지원\n     [이 유] 재판의 집행지휘에 있어서 재판서의 등본 또는 초본의 첨부를 요하는 취지는 개인의 기본적 인권에 중대한 관계가 있는 형집행에 있어 형의 종류 및 범위를 명확키 하기 위하여 재판서의 등본 또는 초본이라고 하는 확실한 증명자료를 첨부시키여 형의 집행에 과오없기를 기하는 것이다. 그러나 재판서 등은 본래 권리가 화체한 유가증권과는 상이하여 형 집행에 절대로 필요한 것이 아니고 다만 증명자료로서 가장 적절하고 또 전형적인 것이므로 통상의 사태를 표준하여 규정한 것이라 할 것이다. 그렇다면 천재지변 등에 인하여 차등 서류의 원본이 멸실되어 등초본등의 작성이 불가능할 경우에는 형의 종류 및 범위를 구체적으로 명확키하기에 족한 타의 증명자료를 첨부하여 재판 집행지휘를 할 수 있다고 해석함이 상당하다 할 것이다. 본건에 있어서 재판서의 원본은 사변으로 인하여 분실된 것인바 여사한 경우에는 전시 소론 서류로써 형의 종류범위를 명확키 하기에 족한 증명 자료로 볼 수 있으니 본건 형집행 이의신청은 이유없다.\n     \n     \n     \n      대법관 백한성(재판장) 오필선 김연수 사광욱 양회경 \n     \n      \n    '

In [157]:
# 찾은 전문 입력
i = 81191
if final.loc[i, '【전문】'] is np.nan:
    final.loc[i, '【전문】'] = add
final.loc[i, '【전문】']

' [신 청 인] 신청인\n     [원 심] 제천지원\n     [이 유] 재판의 집행지휘에 있어서 재판서의 등본 또는 초본의 첨부를 요하는 취지는 개인의 기본적 인권에 중대한 관계가 있는 형집행에 있어 형의 종류 및 범위를 명확키 하기 위하여 재판서의 등본 또는 초본이라고 하는 확실한 증명자료를 첨부시키여 형의 집행에 과오없기를 기하는 것이다. 그러나 재판서 등은 본래 권리가 화체한 유가증권과는 상이하여 형 집행에 절대로 필요한 것이 아니고 다만 증명자료로서 가장 적절하고 또 전형적인 것이므로 통상의 사태를 표준하여 규정한 것이라 할 것이다. 그렇다면 천재지변 등에 인하여 차등 서류의 원본이 멸실되어 등초본등의 작성이 불가능할 경우에는 형의 종류 및 범위를 구체적으로 명확키하기에 족한 타의 증명자료를 첨부하여 재판 집행지휘를 할 수 있다고 해석함이 상당하다 할 것이다. 본건에 있어서 재판서의 원본은 사변으로 인하여 분실된 것인바 여사한 경우에는 전시 소론 서류로써 형의 종류범위를 명확키 하기에 족한 증명 자료로 볼 수 있으니 본건 형집행 이의신청은 이유없다.\n     \n     \n     \n      대법관 백한성(재판장) 오필선 김연수 사광욱 양회경 \n     \n      \n    '

In [364]:
# 전문이 없는 곳에 nan값 입력
noFullContent_idx = final[final['【전문】'].str.len() < 1].index
noFullContent_idx

Int64Index([41993, 42074], dtype='int64')

In [367]:
final.loc[noFullContent_idx, '【전문】'] = np.nan
final.loc[noFullContent_idx, '【전문】']

41993    NaN
42074    NaN
Name: 【전문】, dtype: object

### 이유 결측치 데이터 확인

In [160]:
noWhy = final[final['【이 유】'].isnull()]
noWhy

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목
31,32,NaN,"[채권자, 재항고인] 채권자(소송대리인 변호사 이보현)\n [채무자, 상대...",NaN,민사집행법 제241조 제1항 제2호,"대법원 2013. 9. 16.자 2013마1438 결정(공2013하, 2103)",민사집행법 제241조 제1항 제2호에 따른 매각명령에 대한 즉시항고는 집행력 있는 ...,민사집행법 제241조 제1항 제2호에 따른 매각명령에 대하여 즉시항고를 할 수 있는...,"[대법원 2021. 4. 2., 자, 2020마7789, 결정]",주식특별현금화명령(주식매각명령)
37,38,NaN,"[원고, 상고인] 강음시양명플라스틱유한회사 외 3인 (소송대리인 경인법무법인 담당...",NaN,"[1] 국제사법 제2조 제1항 [2] 국제사법 제2조, 민사소송법 제2조, 제5조 ...","[1] 대법원 2005. 1. 27. 선고 2002다59788 판결(공2005상, ...","[1] 국제사법 제2조 제1항은 ""법원은 당사자 또는 분쟁이 된 사안이 대한민국과 ...",[1] 국제사법 제2조 제1항에서 정한 ‘실질적 관련’의 의미 및 판단 기준[2] ...,"[대법원 2021. 3. 25., 선고, 2018다230588, 판결]",물품대금[대한민국 법원에 국제재판관할권이 인정되는지 여부가 문제된 사건]
38,39,NaN,"[원고, 피상고인] 주식회사 성신건철산업\n [피고, 상고인] 피고 (소송...",NaN,[1] 민사소송법 제173조 제1항 [2] 민사소송법 제173조 제1항,"[1] 대법원 1999. 2. 9. 선고 98다43533 판결, 대법원 2001. ...",[1] 소장부본과 판결정본 등이 공시송달의 방법에 의하여 송달되었다면 특별한 사정이...,[1] 소장부본과 판결정본 등이 공시송달의 방법으로 송달되어 피고가 과실 없이 판결...,"[대법원 2021. 3. 25., 선고, 2020다46601, 판결]",물품대금[추완항소 기간을 도과하였는지 여부가 문제된 사건]
48,49,NaN,"[원고, 피상고인 겸 상고인] 원고 (소송대리인 변호사 박기억)\n [피고...",NaN,"국민건강보험법 제1조, 제53조 제1항, 제57조, 제58조, 사회보장기본법 제3조...","대법원 2002. 1. 8. 선고 2001다40022, 40039 판결(변경), 대...",[다수의견] 국민건강보험공단(이하 ‘공단’이라고 한다)의 손해배상청구권 대위를 인정...,국민건강보험공단이 불법행위의 피해자에게 보험급여를 한 다음 국민건강보험법 제58조 ...,"[대법원 2021. 3. 18., 선고, 2018다287935, 전원합의체 판결]",보험금등청구의소[국민건강보험법에 따른 보험급여를 받은 피해자의 가해자에 대한 치료비...
59,60,NaN,"[원고(선정당사자), 피상고인 겸 상고인] 원고(선정당사자) (소송대리인 변호사 ...",NaN,"민법 제750조, 제751조",NaN,인접 토지에 외벽이 유리로 된 건물 등이 건축되어 과도한 태양반사광이 발생하고 이러...,인접 토지에 외벽이 유리로 된 건물 등이 건축되어 과도한 태양반사광이 발생하고 이러...,"[대법원 2021. 3. 11., 선고, 2013다59142, 판결]",손해배상(기)[인접 건물 외벽의 유리에서 반사되는 강한 태양반사광으로 인한 생활방해...
...,...,...,...,...,...,...,...,...,...,...
79151,79152,NaN,【신 청 인】\n검사\n【피 고 인】\n\n【원심판결】\n부산지방\n【주 문】\n부...,부산지방법원에 본건(부산지방법원 1963. 형고제3554호 피고인 최호원에 대한 특...,"형사소송법 제327조, 군형법 제1조, 군법회의법 제2조 1호",NaN,가. 이등병이 군복무중 1960.12.13. 그 소속대를 무단이탈하므로 1960.3...,"범인이 도망병인 경우, 판결선고 당시 이미 군적에서 삭제되었을 때, 군형법의 적용을...","[대법원 1966. 7. 5., 선고, 63초10, 판결]",재판권쟁의에관한재정신청
79764,79765,NaN,"[원고, 상고인] 원고\n [피고, 피상고인] 피고 1 외 1인\n ...",NaN,NaN,NaN,토지가 소유지가 현존한다는 이유만으로는 그 토지 소유권자가 소유권을 상실한 것이라고...,어느 토지가 유지의 부지로 편입되어 그 유지가 현존한다는 사유만으로 그 토지소유자가...,"[대법원 1964. 12. 29., 선고, 64다1201, 판결]",토지인도
80280,80281,NaN,"[원고(재심원고), 상고인] 원고(재심원고)\n [피고(재심피고), 피상고...",NaN,민사소송법 제422조 제1항 제9호,NaN,확정판결을 하는 상고심이 소송당사자의 원판결에 판단유탈이 있다는 상고이유에 대하여 ...,상고심이 소송당사자의 원판결에 판단유탈이 있다는 주장에 대하여 아무런 언급이 없는 ...,"[대법원 1963. 3. 7., 선고, 62사23, 판결]",건물소유권이전등기말소
80710,80711,NaN,"[원고, 상고인] 원고\n [피고, 피상고인] 인천시장\n [원심판...",NaN,"도시계획령 제35조 제2항, 구 조선시가지계획령 제47조, 소원법 제3조",NaN,"가. 소원법(폐) 제3조 소정의 ""행정처분이 있은 날""이라 함은 환지예정지 지정처분...",가. 토지구획정리에 있어서의 환지예정지 지정처분은 그 토지 소유자에게 통지함으로서 ...,"[대법원 1962. 5. 17., 선고, 62누10, 판결]",환지예정지결정처분취소


In [213]:
import re
p = re.compile("[【\[]+.*[ 이유]+[】\]][\s]*[^【\[]*")

def extract_why(text):
    why = p.search(text).group()
    print(why)

In [214]:
noWhy.iloc[:20].apply(lambda x: extract_why(x['【전문】']), axis=1)

[이 유]재항고이유를 판단한다.
     민사집행법 제241조 제1항 제2호에 따른 매각명령에 대한 즉시항고는 집행력 있는 정본의 유무와 그 송달 여부, 집행개시요건의 존부, 집행장애사유의 존부 등과 같이 매각명령을 할 때 집행법원이 조사하여 준수할 사항에 관한 흠을 이유로 할 수 있을 뿐이고, 집행채권의 소멸 등과 같은 실체상의 사유는 이에 대한 적법한 항고이유가 되지 아니한다(
     대법원 2013. 9. 16.자 2013마1438 결정 등 참조).
      원심은, 채무자가 2020. 7. 20. 채권자를 피공탁자로 7,615,500원을 변제공탁하였고, 채권자가 위 공탁금을 수령한 후 집행법원에 배당포기서를 제출하였음을 인정한 후, 이 사건 집행채권이 변제로 모두 소멸하였으므로 더 이상 강제집행절차를 속행할 수 없다는 이유를 들어, 채무자의 즉시항고를 받아들여 제1심결정을 취소하고 채권자의 이 사건 매각명령 신청을 기각하였다.
      그러나 앞서 본 법리에 비추어 볼 때, 설령 이 사건 집행채권이 변제로 모두 소멸되었다고 하더라도, 이와 같은 실체상의 사유는 이 사건 매각명령에 대한 적법한 즉시항고사유가 될 수 없다. 이 점을 지적하는 재항고이유는 이유 있다.
      그러므로 원심결정을 파기하고, 사건을 원심법원에 환송하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 결정한다.
     
     
     
      대법관 김재형(재판장) 민유숙 이동원(주심) 노태악 
     
      
    
[이 유]상고이유(상고이유서 제출기간이 지난 다음 제출된 준비서면들은 이를 보충하는 범위에서)를 판단한다.
      
      1.  국제재판관할의 판단 기준
     국제사법 제2조 제1항은 "법원은 당사자 또는 분쟁이 된 사안이 대한민국과 실질적 관련이 있는 경우에 국제재판관할권을 가진다. 이 경우 법원은 실질적 관련의 유무를 판단함에 있어 국제재판관할 배분의 이념에 부합하는 합리적인 원칙에 따라야 한다."라고 정하고 있다. ‘

31     None
37     None
38     None
48     None
59     None
67     None
68     None
70     None
72     None
80     None
84     None
85     None
86     None
90     None
93     None
94     None
98     None
109    None
117    None
125    None
dtype: object

In [202]:
for text in noWhy.iloc[:20]['【전문】']: print(text)

 [채권자, 재항고인] 채권자(소송대리인 변호사 이보현)
     [채무자, 상대방] 채무자
     [원심결정] 부산지법 2020. 11. 17.자 2020라2587 결정
     [주 문]
     원심결정을 파기하고, 사건을 부산지방법원에 환송한다.
     
     [이 유]재항고이유를 판단한다.
     민사집행법 제241조 제1항 제2호에 따른 매각명령에 대한 즉시항고는 집행력 있는 정본의 유무와 그 송달 여부, 집행개시요건의 존부, 집행장애사유의 존부 등과 같이 매각명령을 할 때 집행법원이 조사하여 준수할 사항에 관한 흠을 이유로 할 수 있을 뿐이고, 집행채권의 소멸 등과 같은 실체상의 사유는 이에 대한 적법한 항고이유가 되지 아니한다(
     대법원 2013. 9. 16.자 2013마1438 결정 등 참조).
      원심은, 채무자가 2020. 7. 20. 채권자를 피공탁자로 7,615,500원을 변제공탁하였고, 채권자가 위 공탁금을 수령한 후 집행법원에 배당포기서를 제출하였음을 인정한 후, 이 사건 집행채권이 변제로 모두 소멸하였으므로 더 이상 강제집행절차를 속행할 수 없다는 이유를 들어, 채무자의 즉시항고를 받아들여 제1심결정을 취소하고 채권자의 이 사건 매각명령 신청을 기각하였다.
      그러나 앞서 본 법리에 비추어 볼 때, 설령 이 사건 집행채권이 변제로 모두 소멸되었다고 하더라도, 이와 같은 실체상의 사유는 이 사건 매각명령에 대한 적법한 즉시항고사유가 될 수 없다. 이 점을 지적하는 재항고이유는 이유 있다.
      그러므로 원심결정을 파기하고, 사건을 원심법원에 환송하기로 하여, 관여 대법관의 일치된 의견으로 주문과 같이 결정한다.
     
     
     
      대법관 김재형(재판장) 민유숙 이동원(주심) 노태악 
     
      
    
 [원고, 상고인] 강음시양명플라스틱유한회사 외 3인 (소송대리인 경인법무법인 담당변호사 양기오 외 2인)
     [피고, 피상고인] 한미실업 주식회사 (소송대리

In [176]:
noWhy.loc[79151:79153]['【전문】'].str.contains('[이 유]')

79151    True
Name: 【전문】, dtype: bool

### 주문 결측치 데이터 확인

In [310]:
noResult = final[final['【주 문】'].isnull()]
noResult

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목
31,32,NaN,"[채권자, 재항고인] 채권자(소송대리인 변호사 이보현)\n [채무자, 상대...",NaN,민사집행법 제241조 제1항 제2호,"대법원 2013. 9. 16.자 2013마1438 결정(공2013하, 2103)",민사집행법 제241조 제1항 제2호에 따른 매각명령에 대한 즉시항고는 집행력 있는 ...,민사집행법 제241조 제1항 제2호에 따른 매각명령에 대하여 즉시항고를 할 수 있는...,"[대법원 2021. 4. 2., 자, 2020마7789, 결정]",주식특별현금화명령(주식매각명령)
37,38,NaN,"[원고, 상고인] 강음시양명플라스틱유한회사 외 3인 (소송대리인 경인법무법인 담당...",NaN,"[1] 국제사법 제2조 제1항 [2] 국제사법 제2조, 민사소송법 제2조, 제5조 ...","[1] 대법원 2005. 1. 27. 선고 2002다59788 판결(공2005상, ...","[1] 국제사법 제2조 제1항은 ""법원은 당사자 또는 분쟁이 된 사안이 대한민국과 ...",[1] 국제사법 제2조 제1항에서 정한 ‘실질적 관련’의 의미 및 판단 기준[2] ...,"[대법원 2021. 3. 25., 선고, 2018다230588, 판결]",물품대금[대한민국 법원에 국제재판관할권이 인정되는지 여부가 문제된 사건]
38,39,NaN,"[원고, 피상고인] 주식회사 성신건철산업\n [피고, 상고인] 피고 (소송...",NaN,[1] 민사소송법 제173조 제1항 [2] 민사소송법 제173조 제1항,"[1] 대법원 1999. 2. 9. 선고 98다43533 판결, 대법원 2001. ...",[1] 소장부본과 판결정본 등이 공시송달의 방법에 의하여 송달되었다면 특별한 사정이...,[1] 소장부본과 판결정본 등이 공시송달의 방법으로 송달되어 피고가 과실 없이 판결...,"[대법원 2021. 3. 25., 선고, 2020다46601, 판결]",물품대금[추완항소 기간을 도과하였는지 여부가 문제된 사건]
48,49,NaN,"[원고, 피상고인 겸 상고인] 원고 (소송대리인 변호사 박기억)\n [피고...",NaN,"국민건강보험법 제1조, 제53조 제1항, 제57조, 제58조, 사회보장기본법 제3조...","대법원 2002. 1. 8. 선고 2001다40022, 40039 판결(변경), 대...",[다수의견] 국민건강보험공단(이하 ‘공단’이라고 한다)의 손해배상청구권 대위를 인정...,국민건강보험공단이 불법행위의 피해자에게 보험급여를 한 다음 국민건강보험법 제58조 ...,"[대법원 2021. 3. 18., 선고, 2018다287935, 전원합의체 판결]",보험금등청구의소[국민건강보험법에 따른 보험급여를 받은 피해자의 가해자에 대한 치료비...
59,60,NaN,"[원고(선정당사자), 피상고인 겸 상고인] 원고(선정당사자) (소송대리인 변호사 ...",NaN,"민법 제750조, 제751조",NaN,인접 토지에 외벽이 유리로 된 건물 등이 건축되어 과도한 태양반사광이 발생하고 이러...,인접 토지에 외벽이 유리로 된 건물 등이 건축되어 과도한 태양반사광이 발생하고 이러...,"[대법원 2021. 3. 11., 선고, 2013다59142, 판결]",손해배상(기)[인접 건물 외벽의 유리에서 반사되는 강한 태양반사광으로 인한 생활방해...
...,...,...,...,...,...,...,...,...,...,...
81590,81591,제1심 공정에서의 피고인등의 진술 동 증인 공소외 1의 진술의 각 기재 및 성립에 ...,"【상고인, 피고인】\n\n【원심판결】\n제1심 진주지원, 제2심 대구고등\n【이 유...",NaN,형법 제355조제362조,NaN,양도담보권자인 채권자가 자기 채권자인 제3자에게 담보물인 부동산에 관하여 그 소유자...,양도담보로 된 부동산의 소유권이전등기와 불법 영득의 의사,"[대법원 1958. 1. 31., 4290형상404]",횡령
81591,81592,제1심 및 원심공정에서의 피고인의 각 진술을 종합하면 피고인은 공소사실 적시와 여히...,"【상고인, 피고인】\n\n【원심판결】\n제1심 서울지방, 제2심 서울고등\n【이 유...",NaN,형법 제347조,NaN,"피고인이 공소사실적시와 같이 ""갑""으로부터 동인을 제대케하여 준다는 조건으로 금원을...",기망의 의사와 위법수단,"[대법원 1958. 1. 24., 4290형상415]",사기등
81592,81593,경찰 또는 검찰조서라 할지라도 법정증거능력을 구비한 것이면 법원이 조사한 증거와 동...,"【상고인, 피고인】\n\n【원심판결】\n제1심 거창지원, 제2심 대구고등\n【이 유...",NaN,NaN,NaN,검사 또는 사법경찰관의 조서는 법원이 조사한 증거와 동등의 증거가치가 있다,경찰 또는 검찰조서의 증거가치,"[대법원 1958. 1. 24., 4290형상433]",업무상실화
81593,81594,가사 피고인의 신분이 소론과 같이 군적에 있다 할지라도 피고인이 민간에 나와 민간인...,"【상고인, 피고인】\n\n【원심판결】\n제1심 서울지방, 제2심 서울고등\n【이 유...",NaN,"국방경비법 제4조, 형사소송법 제471조, 형법 제35조",NaN,현역군인이 일반범죄를 범한 경우에는 법원 및 군법회의에 각각 관할관이 있다,가. 현후 군인에 대한 법원의 재판권나. 형의 집행정지처분중인 자에 대한 누범가중,"[대법원 1958. 1. 21., 4290형상438]",특수절도등


#### 전문에서 주문 추출

In [330]:
import re
p = re.compile("[【\[]+.*[ 주문심]+[】\]][\s]*(?P<content>[^【\[]*)")

def extract_result(idx, text):
    result = ''
    
    try:
        result = p.search(text).group('content')
    except AttributeError as e:
        print("주문 없음: ", idx)
        
    return result

#### 주문 추출 후 적용
* 주문이 없을 경우 ""

In [343]:
final['【주 문】'] = final.apply(lambda x: extract_result(x.name, x['【전문】']), axis=1)

주문 없음:  3896
주문 없음:  13560
주문 없음:  16348
주문 없음:  30067
주문 없음:  34821
주문 없음:  37452
주문 없음:  37711
주문 없음:  39586
주문 없음:  41993
주문 없음:  42074
주문 없음:  42188
주문 없음:  43665
주문 없음:  44176
주문 없음:  45615
주문 없음:  48503
주문 없음:  48583
주문 없음:  49493
주문 없음:  50471
주문 없음:  50906
주문 없음:  51379
주문 없음:  51794
주문 없음:  53253
주문 없음:  53485
주문 없음:  53591
주문 없음:  54247
주문 없음:  55083
주문 없음:  55144
주문 없음:  55199
주문 없음:  55635
주문 없음:  56378
주문 없음:  56421
주문 없음:  57471
주문 없음:  57817
주문 없음:  58811
주문 없음:  58894
주문 없음:  59120
주문 없음:  60341
주문 없음:  60556
주문 없음:  61443
주문 없음:  61767
주문 없음:  63776
주문 없음:  67922
주문 없음:  70305
주문 없음:  70322
주문 없음:  70342
주문 없음:  70648
주문 없음:  73875
주문 없음:  73876
주문 없음:  74321
주문 없음:  74329
주문 없음:  74369
주문 없음:  74383
주문 없음:  74387
주문 없음:  74429
주문 없음:  74460
주문 없음:  74467
주문 없음:  74563
주문 없음:  74581
주문 없음:  74690
주문 없음:  74721
주문 없음:  74729
주문 없음:  74786
주문 없음:  74852
주문 없음:  74866
주문 없음:  74944
주문 없음:  74962
주문 없음:  74975
주문 없음:  75005
주문 없음:  75047
주문 없음:  75075
주문 없음:  75080
주문 없음: 

* 주문 없는경우 공백에서 nan값으로 변경

In [355]:
noResult_idx = final[final['【주 문】'].str.len() < 1].index
noResult_idx

Int64Index([  296,   799,   813,   874,  1379,  1568,  1579,  2234,  2568,
             3415,
            ...
            81583, 81584, 81586, 81587, 81589, 81590, 81591, 81592, 81593,
            81594],
           dtype='int64', length=859)

In [357]:
for idx in noResult_idx:
    final.loc[idx, '【주 문】'] = np.nan
final[final.index.isin(noResult_idx)]

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목
296,297,1. 항소이유의 요지가. 피고인 1 1) 사실오인 및 법리오해 가) 2018고합29...,【피 고 인】\n피고인 1 외 1인\n【항 소 인】\n피고인 1 및 검사\n【검 사...,NaN,"형법 제355조 제1항, 구 특정경제범죄 가중처벌 등에 관한 법률(2017. 12....",NaN,"甲 주식회사의 실제 운영자인 피고인이 乙로부터 수회에 걸쳐 합계 17억 5,000만...",甲 주식회사의 실제 운영자인 피고인이 乙로부터 사업자금 명목으로 돈을 차용한 후 乙...,"[부산고법 2020. 9. 7., 선고, 2020노52, 판결 : 상고]",특정경제범죄가중처벌등에관한법률위반(배임)(피고인 1에 대하여 예비적 죄명: 배임수재...
799,800,"1. 항소이유의 요지가. 피고인 1, 피고인 2, 피고인 3, 피고인 4, 피고인 ...","【피 고 인】\n피고인 1 외 21인\n【항 소 인】\n피고인 1, 피고인 2, 피...",NaN,NaN,NaN,NaN,NaN,"[인천지방법원 2019. 10. 18., 선고, 2018노4352, 2018노435...",범죄단체조직·범죄단체가입·범죄단체활동·사기·사기방조·자동차관리법위반·위증
813,814,상고이유(상고이유서 제출기간이 경과한 후에 제출된 상고이유보충서 등의 기재는 상고이...,"【피 고 인】\n피고인 1 외 8인\n【상 고 인】\n피고인 1, 피고인 2, 피고...",NaN,"[1] 특정범죄 가중처벌 등에 관한 법률 제8조, 조세범 처벌법 제3조, 상속세 및...","[1] 대법원 2006. 6. 29. 선고 2004도817 판결(공2006하, 14...",NaN,[1] 증여세 납세의무의 성립은 증여재산의 취득시기를 기준으로 하는지 여부(적극) ...,"[대법원 2019. 10. 17., 선고, 2018도16652, 판결]",특정범죄가중처벌등에관한법률위반(조세)·특정경제범죄가중처벌등에관한법률위반(횡령)·특정...
874,875,1. 항소이유의 요지가. 피고인 11) 법리오해 가) 공직선거관리규칙 제59조 제1...,"【피 고 인】\n피고인 1 외 22인\n【항 소 인】\n피고인 1, 피고인 2, 피...",NaN,NaN,NaN,NaN,NaN,"[서울고등법원 2019. 8. 28., 선고, (춘천)2019노115, 판결]",공직선거법위반·정치자금법위반
1379,1380,NaN,"【피 고 인】\n피고인 1 외 10인\n【검 사】\n하동우, 주혜진, 박성진(기소 ...",NaN,NaN,NaN,NaN,NaN,"[부산지방법원 2019. 1. 11., 선고, 2018고합127, 172(병합), ...",특정범죄가중처벌등에관한법률위반(관세)·관세법위반·조세범처벌법위반·특정범죄가중처벌등에...
...,...,...,...,...,...,...,...,...,...,...
81590,81591,제1심 공정에서의 피고인등의 진술 동 증인 공소외 1의 진술의 각 기재 및 성립에 ...,"【상고인, 피고인】\n\n【원심판결】\n제1심 진주지원, 제2심 대구고등\n【이 유...",NaN,형법 제355조제362조,NaN,양도담보권자인 채권자가 자기 채권자인 제3자에게 담보물인 부동산에 관하여 그 소유자...,양도담보로 된 부동산의 소유권이전등기와 불법 영득의 의사,"[대법원 1958. 1. 31., 4290형상404]",횡령
81591,81592,제1심 및 원심공정에서의 피고인의 각 진술을 종합하면 피고인은 공소사실 적시와 여히...,"【상고인, 피고인】\n\n【원심판결】\n제1심 서울지방, 제2심 서울고등\n【이 유...",NaN,형법 제347조,NaN,"피고인이 공소사실적시와 같이 ""갑""으로부터 동인을 제대케하여 준다는 조건으로 금원을...",기망의 의사와 위법수단,"[대법원 1958. 1. 24., 4290형상415]",사기등
81592,81593,경찰 또는 검찰조서라 할지라도 법정증거능력을 구비한 것이면 법원이 조사한 증거와 동...,"【상고인, 피고인】\n\n【원심판결】\n제1심 거창지원, 제2심 대구고등\n【이 유...",NaN,NaN,NaN,검사 또는 사법경찰관의 조서는 법원이 조사한 증거와 동등의 증거가치가 있다,경찰 또는 검찰조서의 증거가치,"[대법원 1958. 1. 24., 4290형상433]",업무상실화
81593,81594,가사 피고인의 신분이 소론과 같이 군적에 있다 할지라도 피고인이 민간에 나와 민간인...,"【상고인, 피고인】\n\n【원심판결】\n제1심 서울지방, 제2심 서울고등\n【이 유...",NaN,"국방경비법 제4조, 형사소송법 제471조, 형법 제35조",NaN,현역군인이 일반범죄를 범한 경우에는 법원 및 군법회의에 각각 관할관이 있다,가. 현후 군인에 대한 법원의 재판권나. 형의 집행정지처분중인 자에 대한 누범가중,"[대법원 1958. 1. 21., 4290형상438]",특수절도등


## 사건번호 처리

In [378]:
final['사건번호'] = final['사건번호'].str.strip('\[\] ')
final.head()

,번호,【이 유】,【전문】,【주 문】,【참조조문】,【참조판례】,【판결요지】,【판시사항】,사건번호,제목
0,1,상고이유를 판단한다. 1. 헌법불합치결정과 잠정적용의 범위구 형사소송법(2019...,【피 고 인】\n피고인\n【상 고 인】\n검사\n【변 호 인】\n법무법인 여는 담당...,상고를 기각한다.\n,"헌법 제16조, 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되...","대법원 2011. 9. 29. 선고 2008두18885 판결(공2011하, 2234...",(가) 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되기 전의 ...,헌법재판소가 구 형사소송법 제216조 제1항 제1호 중 제200조의2에 관한 부분(...,"대법원 2021. 5. 27., 선고, 2018도13458, 판결",특수공무집행방해치상ㆍ특수공무집행방해
1,2,상고이유를 판단한다. 1. 상고이유 제1점에 관하여 가. 「상가건물 임대차보호...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 명경 담당변호사 김재윤...",상고를 모두 기각한다. 상고비용은 피고가 부담한다.\n,"[1] 상가건물 임대차보호법 제10조 제1항 제1호, 제10조의8 [2] 민법 제1...",[1] 대법원 2014. 7. 24. 선고 2012다58975 판결 / [2] 대법...,[1] 상가건물 임대차보호법(이하 ‘상가임대차법’이라고 한다) 제10조의8은 임대인...,[1] 상가건물 임대차보호법의 적용을 받는 상가건물의 임대차기간 중 어느 때라도 차...,"대법원 2021. 5. 13., 선고, 2020다255429, 판결",건물명도(인도)
2,3,상고이유를 판단한다. 1. 원고의 상고이유에 대하여 가. 이 사건 RG보험 ...,"【원고, 상고인 겸 피상고인】\n유한회사 좋은기업지배구조연구소 (소송대리인 법무법인...","상고를 모두 기각한다. 상고비용 중 원고의 상고로 인한 부분은 원고가 부담하고, 피...","상법 제403조 제1항, 제2항, 제3항","대법원 2010. 4. 15. 선고 2009다98058 판결, 대법원 2018. 2...","상법 제403조 제1항, 제2항, 제3항은 ‘발행주식 총수의 100분의 1 이상에 ...",상법 제403조 제2항에 따른 서면에 기재되어야 하는 ‘이유’에 책임추궁 대상 이사...,"대법원 2021. 5. 13., 선고, 2019다291399, 판결",손해배상(기)
3,4,상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 범위...,"【원고, 상고인】\n원고 (소송대리인 법무법인 중추 담당변호사 김선흠 외 3인)\n...","원심판결을 파기하고, 사건을 서울고등법원에 환송한다.\n","[1] 민법 제105조, 제390조, 제543조, 제551조, 민사소송법 제288조...","[1] 대법원 1989. 4. 25. 선고 86다카1147, 1148 판결(공198...",[1] 계약이 합의에 따라 해제되거나 해지된 경우에는 상대방에게 손해배상을 하기로 ...,"[1] 계약이 합의에 따라 해제되거나 해지된 경우, 채무불이행으로 인한 손해배상을 ...","대법원 2021. 5. 7., 선고, 2017다220416, 판결",대여금
4,5,"상고이유를 판단한다. 1. 원심의 인정에 따르면, 이 사건 부동산에 관하여 ‘종...","【원고, 피상고인】\n대미륵봉심회 (소송대리인 법무법인 파랑 담당변호사 이형재 외 ...","원심판결을 파기하고, 사건을 대전고등법원에 환송한다.\n","민법 제214조, 부동산등기법 제52조 제1호, 민사소송법 제288조","대법원 1993. 10. 8. 선고 93다28867 판결(공1993하, 3050),...",등기명의인의 표시변경 또는 경정의 부기등기가 등기명의인의 동일성을 해치는 방법으로 ...,등기명의인의 표시변경 또는 경정의 부기등기에 의하여 부동산등기사항증명서상의 표시가 ...,"대법원 2021. 5. 7., 선고, 2020다299214, 판결",등기명의인표시변경등기말소


In [380]:
final.columns = ['번호', '이유', '전문', '주문', '참조조문', '참조판례', '판결요지', '판시사항', '사건번호', '제목']
final.head()

,번호,이유,전문,주문,참조조문,참조판례,판결요지,판시사항,사건번호,제목
0,1,상고이유를 판단한다. 1. 헌법불합치결정과 잠정적용의 범위구 형사소송법(2019...,【피 고 인】\n피고인\n【상 고 인】\n검사\n【변 호 인】\n법무법인 여는 담당...,상고를 기각한다.\n,"헌법 제16조, 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되...","대법원 2011. 9. 29. 선고 2008두18885 판결(공2011하, 2234...",(가) 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되기 전의 ...,헌법재판소가 구 형사소송법 제216조 제1항 제1호 중 제200조의2에 관한 부분(...,"대법원 2021. 5. 27., 선고, 2018도13458, 판결",특수공무집행방해치상ㆍ특수공무집행방해
1,2,상고이유를 판단한다. 1. 상고이유 제1점에 관하여 가. 「상가건물 임대차보호...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 명경 담당변호사 김재윤...",상고를 모두 기각한다. 상고비용은 피고가 부담한다.\n,"[1] 상가건물 임대차보호법 제10조 제1항 제1호, 제10조의8 [2] 민법 제1...",[1] 대법원 2014. 7. 24. 선고 2012다58975 판결 / [2] 대법...,[1] 상가건물 임대차보호법(이하 ‘상가임대차법’이라고 한다) 제10조의8은 임대인...,[1] 상가건물 임대차보호법의 적용을 받는 상가건물의 임대차기간 중 어느 때라도 차...,"대법원 2021. 5. 13., 선고, 2020다255429, 판결",건물명도(인도)
2,3,상고이유를 판단한다. 1. 원고의 상고이유에 대하여 가. 이 사건 RG보험 ...,"【원고, 상고인 겸 피상고인】\n유한회사 좋은기업지배구조연구소 (소송대리인 법무법인...","상고를 모두 기각한다. 상고비용 중 원고의 상고로 인한 부분은 원고가 부담하고, 피...","상법 제403조 제1항, 제2항, 제3항","대법원 2010. 4. 15. 선고 2009다98058 판결, 대법원 2018. 2...","상법 제403조 제1항, 제2항, 제3항은 ‘발행주식 총수의 100분의 1 이상에 ...",상법 제403조 제2항에 따른 서면에 기재되어야 하는 ‘이유’에 책임추궁 대상 이사...,"대법원 2021. 5. 13., 선고, 2019다291399, 판결",손해배상(기)
3,4,상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 범위...,"【원고, 상고인】\n원고 (소송대리인 법무법인 중추 담당변호사 김선흠 외 3인)\n...","원심판결을 파기하고, 사건을 서울고등법원에 환송한다.\n","[1] 민법 제105조, 제390조, 제543조, 제551조, 민사소송법 제288조...","[1] 대법원 1989. 4. 25. 선고 86다카1147, 1148 판결(공198...",[1] 계약이 합의에 따라 해제되거나 해지된 경우에는 상대방에게 손해배상을 하기로 ...,"[1] 계약이 합의에 따라 해제되거나 해지된 경우, 채무불이행으로 인한 손해배상을 ...","대법원 2021. 5. 7., 선고, 2017다220416, 판결",대여금
4,5,"상고이유를 판단한다. 1. 원심의 인정에 따르면, 이 사건 부동산에 관하여 ‘종...","【원고, 피상고인】\n대미륵봉심회 (소송대리인 법무법인 파랑 담당변호사 이형재 외 ...","원심판결을 파기하고, 사건을 대전고등법원에 환송한다.\n","민법 제214조, 부동산등기법 제52조 제1호, 민사소송법 제288조","대법원 1993. 10. 8. 선고 93다28867 판결(공1993하, 3050),...",등기명의인의 표시변경 또는 경정의 부기등기가 등기명의인의 동일성을 해치는 방법으로 ...,등기명의인의 표시변경 또는 경정의 부기등기에 의하여 부동산등기사항증명서상의 표시가 ...,"대법원 2021. 5. 7., 선고, 2020다299214, 판결",등기명의인표시변경등기말소


In [384]:
final = final[['번호', '제목', '사건번호', '판시사항', '판결요지', '참조판례', '참조조문', '주문', '이유', '전문']]
final.head()

,번호,제목,사건번호,판시사항,판결요지,참조판례,참조조문,주문,이유,전문
0,1,특수공무집행방해치상ㆍ특수공무집행방해,"대법원 2021. 5. 27., 선고, 2018도13458, 판결",헌법재판소가 구 형사소송법 제216조 제1항 제1호 중 제200조의2에 관한 부분(...,(가) 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되기 전의 ...,"대법원 2011. 9. 29. 선고 2008두18885 판결(공2011하, 2234...","헌법 제16조, 구 형사소송법(2019. 12. 31. 법률 제16850호로 개정되...",상고를 기각한다.\n,상고이유를 판단한다. 1. 헌법불합치결정과 잠정적용의 범위구 형사소송법(2019...,【피 고 인】\n피고인\n【상 고 인】\n검사\n【변 호 인】\n법무법인 여는 담당...
1,2,건물명도(인도),"대법원 2021. 5. 13., 선고, 2020다255429, 판결",[1] 상가건물 임대차보호법의 적용을 받는 상가건물의 임대차기간 중 어느 때라도 차...,[1] 상가건물 임대차보호법(이하 ‘상가임대차법’이라고 한다) 제10조의8은 임대인...,[1] 대법원 2014. 7. 24. 선고 2012다58975 판결 / [2] 대법...,"[1] 상가건물 임대차보호법 제10조 제1항 제1호, 제10조의8 [2] 민법 제1...",상고를 모두 기각한다. 상고비용은 피고가 부담한다.\n,상고이유를 판단한다. 1. 상고이유 제1점에 관하여 가. 「상가건물 임대차보호...,"【원고, 피상고인】\n원고 1 외 1인 (소송대리인 법무법인 명경 담당변호사 김재윤..."
2,3,손해배상(기),"대법원 2021. 5. 13., 선고, 2019다291399, 판결",상법 제403조 제2항에 따른 서면에 기재되어야 하는 ‘이유’에 책임추궁 대상 이사...,"상법 제403조 제1항, 제2항, 제3항은 ‘발행주식 총수의 100분의 1 이상에 ...","대법원 2010. 4. 15. 선고 2009다98058 판결, 대법원 2018. 2...","상법 제403조 제1항, 제2항, 제3항","상고를 모두 기각한다. 상고비용 중 원고의 상고로 인한 부분은 원고가 부담하고, 피...",상고이유를 판단한다. 1. 원고의 상고이유에 대하여 가. 이 사건 RG보험 ...,"【원고, 상고인 겸 피상고인】\n유한회사 좋은기업지배구조연구소 (소송대리인 법무법인..."
3,4,대여금,"대법원 2021. 5. 7., 선고, 2017다220416, 판결","[1] 계약이 합의에 따라 해제되거나 해지된 경우, 채무불이행으로 인한 손해배상을 ...",[1] 계약이 합의에 따라 해제되거나 해지된 경우에는 상대방에게 손해배상을 하기로 ...,"[1] 대법원 1989. 4. 25. 선고 86다카1147, 1148 판결(공198...","[1] 민법 제105조, 제390조, 제543조, 제551조, 민사소송법 제288조...","원심판결을 파기하고, 사건을 서울고등법원에 환송한다.\n",상고이유(상고이유서 제출기간이 지난 다음 제출된 상고이유보충서는 이를 보충하는 범위...,"【원고, 상고인】\n원고 (소송대리인 법무법인 중추 담당변호사 김선흠 외 3인)\n..."
4,5,등기명의인표시변경등기말소,"대법원 2021. 5. 7., 선고, 2020다299214, 판결",등기명의인의 표시변경 또는 경정의 부기등기에 의하여 부동산등기사항증명서상의 표시가 ...,등기명의인의 표시변경 또는 경정의 부기등기가 등기명의인의 동일성을 해치는 방법으로 ...,"대법원 1993. 10. 8. 선고 93다28867 판결(공1993하, 3050),...","민법 제214조, 부동산등기법 제52조 제1호, 민사소송법 제288조","원심판결을 파기하고, 사건을 대전고등법원에 환송한다.\n","상고이유를 판단한다. 1. 원심의 인정에 따르면, 이 사건 부동산에 관하여 ‘종...","【원고, 피상고인】\n대미륵봉심회 (소송대리인 법무법인 파랑 담당변호사 이형재 외 ..."


## 기타 및 저장

In [385]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82315 entries, 0 to 82314
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   번호      82315 non-null  int64 
 1   제목      82297 non-null  object
 2   사건번호    82315 non-null  object
 3   판시사항    70650 non-null  object
 4   판결요지    62422 non-null  object
 5   참조판례    38465 non-null  object
 6   참조조문    69375 non-null  object
 7   주문      81456 non-null  object
 8   이유      68748 non-null  object
 9   전문      82313 non-null  object
dtypes: int64(1), object(9)
memory usage: 6.3+ MB


In [386]:
final.to_csv("Judgement_Data_All_final.csv", encoding='utf-8-sig')